In [ ]:
import pickle
import os
import datetime
from bs4 import BeautifulSoup
import requests
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import pandas as pd
import numpy as np

In [ ]:
from datetime import datetime

In [ ]:
from google.colab import drive
import os

In [ ]:
start_date = '2024-08-01'
end_date = '2024-08-30'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define the folder path in your Google Drive
#folder_path = '/content/drive/MyDrive/CAPSTONE July 2023-2024'
folder_path = '/content/drive/MyDrive/NLP Assignment(Anup)'

In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
}

def scrape_url(url):
  content = requests.get(url).text
  bsObj = BeautifulSoup(content, features="html.parser")
  try:
    news = bsObj.find('div', attrs={'class': "_s30J clearfix"}).text
    headline = bsObj.find('h1', attrs={'class': 'HNMDR'}).text
    return url, headline, news
  except Exception as e:
    print(f"Error scraping URL {url}: {e}")
    return np.nan, url, np.nan, np.nan

def extract_news(start_date, end_date):
  start = datetime.strptime(start_date, "%Y-%m-%d").toordinal() - datetime(1900, 1, 1).toordinal() + 2
  end = datetime.strptime(end_date, "%Y-%m-%d").toordinal() - datetime(1900, 1, 1).toordinal() + 2

  news_urls = []
  for dt in range(start, end + 1):

    date = datetime.fromordinal(dt + datetime(1900, 1, 1).toordinal() - 2)
    year = date.year
    month = date.month
    day = date.day
    url = f'https://timesofindia.indiatimes.com/{year}/{month}/{day}/archivelist/year-{year},month-{month},starttime-{dt}.cms'
    print(url)
    doc = requests.get(url).text
    bsObj = BeautifulSoup(doc, features="html.parser")
    contents = bsObj.findAll('tr', attrs={'class': 'rightColWrap'})
    for content in contents:
      links = content.findAll(name='a')
      for link in links:
        news_link = link.get_attribute_list('href')[0]
        news_urls.append(news_link)

    # Scrape news content asynchronously
  scraped_news = []
  with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(scrape_url, url) for url in news_urls]
    for future in tqdm(as_completed(futures), total=len(futures), desc="Scraping News"):
      result = future.result()
      scraped_news.append(result)

    # Save scraped news to pickle file
  file_name = os.path.join(folder_path, f'{start}_{end}.pkl')
  with open(file_name, 'wb') as file:
    pickle.dump(scraped_news, file, protocol=pickle.HIGHEST_PROTOCOL)
    return scraped_news

def run_extraction():
    try:

        extract_news(start_date, end_date)

        print("Extraction Complete", "News extraction process completed successfully.")

    except Exception as e:
        print("Error", f"An error occurred: {str(e)}")






In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
}

def scrape_url(url):
    try:
        content = requests.get(url, headers=headers).text
        bsObj = BeautifulSoup(content, features="html.parser")

        # Check for premium content indicator, if present, skip the article
        if bsObj.find('div', attrs={'class': 'premium-content-class-name'}):
            print(f"Premium article encountered at {url}, skipping...")
            return url, np.nan, np.nan  # Returning NaN for premium articles

        news = bsObj.find('div', attrs={'class': "_s30J clearfix"}).text
        headline = bsObj.find('h1', attrs={'class': 'HNMDR'}).text

        return url, headline, news

    except Exception as e:
        print(f"Error scraping URL {url}: {e}")
        return url, np.nan, np.nan  # Return NaN if any error occurs


def extract_news(start_date, end_date):
  start = datetime.strptime(start_date, "%Y-%m-%d").toordinal() - datetime(1900, 1, 1).toordinal() + 2
  end = datetime.strptime(end_date, "%Y-%m-%d").toordinal() - datetime(1900, 1, 1).toordinal() + 2

  news_urls = []
  for dt in range(start, end + 1):

    date = datetime.fromordinal(dt + datetime(1900, 1, 1).toordinal() - 2)
    year = date.year
    month = date.month
    day = date.day
    url = f'https://timesofindia.indiatimes.com/{year}/{month}/{day}/archivelist/year-{year},month-{month},starttime-{dt}.cms'
    print(url)
    doc = requests.get(url).text
    bsObj = BeautifulSoup(doc, features="html.parser")
    contents = bsObj.findAll('tr', attrs={'class': 'rightColWrap'})
    for content in contents:
      links = content.findAll(name='a')
      for link in links:
        news_link = link.get_attribute_list('href')[0]
        news_urls.append(news_link)

    # Scrape news content asynchronously
  scraped_news = []
  with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(scrape_url, url) for url in news_urls]
    for future in tqdm(as_completed(futures), total=len(futures), desc="Scraping News"):
      result = future.result()
      scraped_news.append(result)

    # Save scraped news to pickle file
  file_name = os.path.join(folder_path, f'{start}_{end}.pkl')
  with open(file_name, 'wb') as file:
    pickle.dump(scraped_news, file, protocol=pickle.HIGHEST_PROTOCOL)
    return scraped_news

def run_extraction():
    try:

        extract_news(start_date, end_date)

        print("Extraction Complete", "News extraction process completed successfully.")

    except Exception as e:
        print("Error", f"An error occurred: {str(e)}")






In [ ]:
run_extraction()

https://timesofindia.indiatimes.com/2024/8/1/archivelist/year-2024,month-8,starttime-45505.cms
https://timesofindia.indiatimes.com/2024/8/2/archivelist/year-2024,month-8,starttime-45506.cms
https://timesofindia.indiatimes.com/2024/8/3/archivelist/year-2024,month-8,starttime-45507.cms
https://timesofindia.indiatimes.com/2024/8/4/archivelist/year-2024,month-8,starttime-45508.cms
https://timesofindia.indiatimes.com/2024/8/5/archivelist/year-2024,month-8,starttime-45509.cms
https://timesofindia.indiatimes.com/2024/8/6/archivelist/year-2024,month-8,starttime-45510.cms
https://timesofindia.indiatimes.com/2024/8/7/archivelist/year-2024,month-8,starttime-45511.cms
https://timesofindia.indiatimes.com/2024/8/8/archivelist/year-2024,month-8,starttime-45512.cms
https://timesofindia.indiatimes.com/2024/8/9/archivelist/year-2024,month-8,starttime-45513.cms
https://timesofindia.indiatimes.com/2024/8/10/archivelist/year-2024,month-8,starttime-45514.cms
https://timesofindia.indiatimes.com/2024/8/11/arc

Scraping News:  70%|███████   | 10566/15000 [18:38<00:45, 97.46it/s]

Error scraping URL /life-style/health-fitness/weight-loss/how-much-weight-should-one-lose-in-a-month/articleshow/112690585.cms: Invalid URL '/life-style/health-fitness/weight-loss/how-much-weight-should-one-lose-in-a-month/articleshow/112690585.cms': No scheme supplied. Perhaps you meant https:///life-style/health-fitness/weight-loss/how-much-weight-should-one-lose-in-a-month/articleshow/112690585.cms?Error scraping URL /entertainment/kannada/movies/news/its-challenging-to-write-comedy-says-janardhan-chikkanna/articleshow/112680388.cms: Invalid URL '/entertainment/kannada/movies/news/its-challenging-to-write-comedy-says-janardhan-chikkanna/articleshow/112680388.cms': No scheme supplied. Perhaps you meant https:///entertainment/kannada/movies/news/its-challenging-to-write-comedy-says-janardhan-chikkanna/articleshow/112680388.cms?

Error scraping URL /astrology/horoscope/horoscope-today-august-22-2024-read-your-todays-astrological-predictions/articleshow/112683698.cms: Invalid URL '/astr

Scraping News:  71%|███████   | 10635/15000 [18:38<00:22, 197.60it/s]

Error scraping URL /world/uk/queen-elizabeth-ii-found-trumps-to-be-gracious-guests-says-palace-official/articleshow/112691573.cms: Invalid URL '/world/uk/queen-elizabeth-ii-found-trumps-to-be-gracious-guests-says-palace-official/articleshow/112691573.cms': No scheme supplied. Perhaps you meant https:///world/uk/queen-elizabeth-ii-found-trumps-to-be-gracious-guests-says-palace-official/articleshow/112691573.cms?Error scraping URL /city/chandigarh/bjp-wont-let-a-congress-government-in-haryana-last-state-minister/articleshow/112692504.cms: Invalid URL '/city/chandigarh/bjp-wont-let-a-congress-government-in-haryana-last-state-minister/articleshow/112692504.cms': No scheme supplied. Perhaps you meant https:///city/chandigarh/bjp-wont-let-a-congress-government-in-haryana-last-state-minister/articleshow/112692504.cms?
Error scraping URL /astrology/horoscope/sagittarius-daily-horoscope-today-august-22-2024-day-to-unwind-and-let-go-of-your-worries/articleshow/112677410.cms: Invalid URL '/astrol

...

Error scraping URL /city/delhi/government-appoints-interim-heads-of-ioc-and-hpcl/articleshow/112878070.cms: Invalid URL '/city/delhi/government-appoints-interim-heads-of-ioc-and-hpcl/articleshow/112878070.cms': No scheme supplied. Perhaps you meant https:///city/delhi/government-appoints-interim-heads-of-ioc-and-hpcl/articleshow/112878070.cms?
Error scraping URL /city/chennai/iit-m-and-iit-ropar-collaboration-for-data-science-students/articleshow/112878336.cms: Invalid URL '/city/chennai/iit-m-and-iit-ropar-collaboration-for-data-science-students/articleshow/112878336.cms': No scheme supplied. Perhaps you meant https:///city/chennai/iit-m-and-iit-ropar-collaboration-for-data-science-students/articleshow/112878336.cms?
Error scraping URL /city/chennai/police-seek-help-in-nabbing-train-assault-suspect/articleshow/112878344.cms: Invalid URL '/city/chennai/police-seek-help-in-nabbing-train-assault-suspect/articleshow/112878344.cms': No scheme supplied. Perhaps you meant https:///city/chenn

Scraping News:  99%|█████████▊| 14782/15000 [18:42<00:00, 1133.78it/s]

Error scraping URL /city/lucknow/bsip-museum-to-take-you-back-to-dino-era/articleshow/112880169.cms: Invalid URL '/city/lucknow/bsip-museum-to-take-you-back-to-dino-era/articleshow/112880169.cms': No scheme supplied. Perhaps you meant https:///city/lucknow/bsip-museum-to-take-you-back-to-dino-era/articleshow/112880169.cms?
Error scraping URL /city/bhopal/12-year-old-girl-escapes-rape-bid-in-mp-hospital-accused-held/articleshow/112880173.cms: Invalid URL '/city/bhopal/12-year-old-girl-escapes-rape-bid-in-mp-hospital-accused-held/articleshow/112880173.cms': No scheme supplied. Perhaps you meant https:///city/bhopal/12-year-old-girl-escapes-rape-bid-in-mp-hospital-accused-held/articleshow/112880173.cms?
Error scraping URL /city/ranchi/jhalsa-holds-camps-for-differently-abled-kids/articleshow/112880177.cms: Invalid URL '/city/ranchi/jhalsa-holds-camps-for-differently-abled-kids/articleshow/112880177.cms': No scheme supplied. Perhaps you meant https:///city/ranchi/jhalsa-holds-camps-for-dif

Scraping News: 100%|██████████| 15000/15000 [18:42<00:00, 13.36it/s]  


Error scraping URL /city/nagpur/shivaji-maharaj-statue-sculptor-linked-to-rss/articleshow/112908364.cms: Invalid URL '/city/nagpur/shivaji-maharaj-statue-sculptor-linked-to-rss/articleshow/112908364.cms': No scheme supplied. Perhaps you meant https:///city/nagpur/shivaji-maharaj-statue-sculptor-linked-to-rss/articleshow/112908364.cms?
Error scraping URL /city/kolkata/kilburn-engineering-ltd-to-raise-300-crore-through-preferential-allotment/articleshow/112908367.cms: Invalid URL '/city/kolkata/kilburn-engineering-ltd-to-raise-300-crore-through-preferential-allotment/articleshow/112908367.cms': No scheme supplied. Perhaps you meant https:///city/kolkata/kilburn-engineering-ltd-to-raise-300-crore-through-preferential-allotment/articleshow/112908367.cms?
Error scraping URL /city/chandigarh/haryana-govt-seeks-cancellation-of-bail-for-congress-mla-mamman-khan-in-nuh-riots/articleshow/112908377.cms: Invalid URL '/city/chandigarh/haryana-govt-seeks-cancellation-of-bail-for-congress-mla-mamman-

In [ ]:
# Load the "news.pkl" file into a DataFrame
news_df = pd.read_pickle("/content/drive/MyDrive/NLP Assignment(Anup)/45505_45534.pkl")

In [ ]:
news_df_ = pd.DataFrame(news_df)

In [ ]:
news_df_

,0,1,2
0,http://timesofindia.indiatimes.com//sports/cri...,IPL owners' meeting: Different views emerge ov...,"MUMBAI: As expected, there was plenty of discu..."
1,http://timesofindia.indiatimes.com//sports/par...,"After Rio high, Indian gymnastics crash lands ...","In 2016, when Dipa Karmakar finished fourth at..."
2,http://timesofindia.indiatimes.com//technology...,Flying to the US: What you need to know about ...,"In two separate incidents, unsuspecting travel..."
3,http://timesofindia.indiatimes.com//world/us/f...,Fed hints at rate cut as inflation nears 2% ta...,The Federal Reserve on Wednesday indicated tha...
4,http://timesofindia.indiatimes.com//india/delh...,Delhi schools closed on Thursday due to heavy ...,NEW DELHI: Delhi education minister Atishi ann...
...,...,...,...
14995,/city/jaipur/2-pilgrims-die-in-suv-accident/ar...,NaN,NaN
14996,/city/jaipur/rajasthan-high-court-stays-arrest...,NaN,NaN
14997,/city/chandigarh/panjab-university-student-cou...,NaN,NaN
14998,/city/jaipur/man-dies-after-slipping-into-cana...,NaN,NaN


In [ ]:
news_df_ = news_df_.rename(columns={0: 'url', 1: 'headline', 2: 'news'})


In [ ]:
news_df_

,url,headline,news
0,http://timesofindia.indiatimes.com//sports/cri...,IPL owners' meeting: Different views emerge ov...,"MUMBAI: As expected, there was plenty of discu..."
1,http://timesofindia.indiatimes.com//sports/par...,"After Rio high, Indian gymnastics crash lands ...","In 2016, when Dipa Karmakar finished fourth at..."
2,http://timesofindia.indiatimes.com//technology...,Flying to the US: What you need to know about ...,"In two separate incidents, unsuspecting travel..."
3,http://timesofindia.indiatimes.com//world/us/f...,Fed hints at rate cut as inflation nears 2% ta...,The Federal Reserve on Wednesday indicated tha...
4,http://timesofindia.indiatimes.com//india/delh...,Delhi schools closed on Thursday due to heavy ...,NEW DELHI: Delhi education minister Atishi ann...
...,...,...,...
14995,/city/jaipur/2-pilgrims-die-in-suv-accident/ar...,NaN,NaN
14996,/city/jaipur/rajasthan-high-court-stays-arrest...,NaN,NaN
14997,/city/chandigarh/panjab-university-student-cou...,NaN,NaN
14998,/city/jaipur/man-dies-after-slipping-into-cana...,NaN,NaN


In [ ]:
news_df_.isna().sum()

,0
url,0
headline,4500
news,4500
